# Unit 5 - Financial Planning

In [1]:
## the initial environment setup wasn't really working, so i restarted the system multiple times and each time the jupyter lab changed the outcome bugs
## in which I put the codes as what the class activities for references
## 12/12/22 EDITED

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import json
import sys
## import json and sys for hide .env file
%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1
my_eth = 1

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [ ]:
## sources: https://deallen7.medium.com/how-to-create-a-pandas-dataframe-from-an-api-endpoint-in-a-jupyter-notebook-f2561f766ca3
# Fetch current BTC price
my_btc_value = requests.get(btc_url)
my_btc_value = my_btc_value.json()
my_btc_value = my_btc_value['data']['1']['quotes']['USD']['price']
my_btc_value = my_btc * my_btc_value
my_btc_value
# Fetch current ETH price
my_eth_value = requests.get(eth_url)
my_eth_value = my_eth_value.json()
my_eth_value = my_eth_value['data']['1027']['quotes']['USD']['price']
my_eth_value = my_eth * my_eth_value
my_eth_value
# Compute current value of my crpto
my_crypto_value = my_btc_value + my_eth_value

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:

# Format current date as ISO format
today = pd.Timestamp("2022-12-08", tz="America/New_York").isoformat()
today
print(today)
# Set the tickers
tickers = ["AGG", "SPY"]
tickers
# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
df_profolio = alpaca_api.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df


# Reorganize the DataFrame
# Separate ticker data
AGG = df_portfolio[df_portfolio['symbol']=='SPY'].drop("symbol", axis=1)
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop("symbol", axis=1)


# Concatenate the ticker DataFrames
df_profolio = pd.concat([AGG,SPY], axis = 1, keys = ['AGG', 'SPY'])

# Preview DataFrame
df_profolio

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = df_profolio['AGG']['close']
spy_close_price = df_profolio['SPY']['close'] 

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
my_spy_value = my_agg * agg_close_price
my_agg_value = my_spy * spy_close_price
# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = MCSimulation(
    profolio_data = df_profolio,
    weights = [.60, .40],
    num_simulation = 500,
    num_income_days = 21
)

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
df_savings = df.DataFrame(
    monthly_income,
    tichers,
    dataframe,
    ##not sure if the structure is correct, need more time to review the build of a df
)

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
df_savings.barplot()

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = emergency_fund + df_savings['close']

# Validate saving health
total_savings.head()

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
df_ticker = alpaca_api.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
AGG = df_ticker[df_ticker['symbol']=='AGG'].drop('symbol',axis=1)
SPY = df_ticker[df_ticker['symbol']=='SPY'].drop('symbol',axis=1)



# Concatenate the ticker DataFrames
df_ticker = pd.concat([AGG, SPY], axis = 1, keys = ["AGG","SPY"])

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirtyyear = MCSimulation(
    profolio_data = df_ticker,
    weights = [.60,.40],
    num_simulation = 500,
    num_trading_days = 252*30
)

In [ ]:
# Printing the simulation input data
MC_thirtyyear.profolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_thirtyyear.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot = MC_thirtyyear.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_thirtyyear.plot_distrubution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
even_tbl = MC_thirtyyear.summarize_cumulative_return()

# Print summary statistics
print(even_tbl)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(even_tbl[8]*20000,2)
ci_upper = round(even_tbl[9]*20000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(even_tbl[8]*30000,2)
ci_upper = round(even_tbl[9]*30000,2)
# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")